## Building better prompts

An interesting window is open in most of the LLMs for tuning chat completions for usefulness.
We begin with the basics of a Chat Completions API, and progress through prompt engineering tactics.

## Import library(-ies)

In [17]:
# Source keys from an .env
from dotenv import load_dotenv
# Source operating system resources
import os
# Access libraries for AI  
from openai import OpenAI
# Standard library for API requests
import requests
# Enable working with Unix timestamps
from datetime import datetime
# Enable working with json
import json
# Enable display of html responses
from IPython.display import display, HTML, Markdown, Latex



In [2]:
# Load environment variables
load_dotenv()
api_key = os.getenv('PPLX_API_KEY')

## Authenticate

In [3]:
YOUR_API_KEY = api_key
base_url = "https://api.perplexity.ai"

## Keep it DRY

In [4]:
# Choose model
model = "llama-3-sonar-large-32k-online"

# Create a client
client = OpenAI(api_key=YOUR_API_KEY, base_url="https://api.perplexity.ai")

In [5]:
# Make a request
def get_completion(user_prompt, system_role, model):
    messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_prompt}
    ]

    # chat completion without streaming
    response = client.chat.completions.create(
        model=model,
        messages=messages,
    )
    # Print the message content
    return response

In [6]:
# Accomodate common formats
def get_completion_json(system_role, user_prompt, model):
    # Note authentication is configured above

    
    # chat completion without streaming
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        response_format={"type": "json_object"}
    )
                         
    # Print the message content
    return response

In [7]:
# Print the content of the response
def summarize_message(from_the_ai):
    # Summarize the message
    print(f"The {response.choices[0].message.role} role finished \n♢ with \"{response.choices[0].finish_reason}\" at: {datetime.fromtimestamp(response.created)}")
    print(f"The model used was: {response.model}")
    print(f"♢ The message was: \n\n{response.choices[0].message.content}")

In [15]:
# Pretty print HTML response
def pretty_print_html(from_the_ai):
    display(HTML(response.choices[0].message.content))

In [14]:
# Pretty print Markdown response
def pretty_print_markdown(from_the_ai):
    display(Markdown(response.choices[0].message.content))

## Basic concepts

1. Use the latest models. For perplexity.ai API there is a [list](https://docs.perplexity.ai/docs/model-cards):  
   ```llama-3-sonar-large-32k-online```  
   We can use open-source:    
   ```mixtral-8x7b-instruct	```  
2. Instructions must be clear and specific
3. Instruct at the beginning of the prompt
4. Delimit distinct parts; The basic delimiters are ⎛ \`\`\`{three back-ticks}\`\`\` ⎞ ⎛ \"\"\"{quotes}\"\"\" ⎞ ⎛ {curly braces} ⎞
5. Get to know how variables work  

## Better instructions

5. Precise, __descriptive__, with sufficient detail

In [8]:
# define a better prompt
system_role = '''
you are an experienced entrepreneur. 
you only make investments which are probably and likely profitable.
your access to intelligence is comprehensive, so you present complex concepts
in college level English with clear structure and in concise, simple terms.
you never need to apologize and are forthright in presenting the facts.
you are are telling only the truth of the source data available to you.
You review your responses to avoid repetition, and explain logically how they were generated.
You advise as an entrepreneur with experience building self-sustaining businesses.
'''

text = '''
What are the best five ways to make the first $10000 dollars on Faire?
Considerations:
1. You have really clear rationale in using Faire to build a business
2. You support your ideas with the three detailed steps you would take next based on research
3. Take time to review your answer before responding
4. How much extra time is needed for your review
'''

prompt = f'''
Analyze the text below fewer than 500 words
Text: ```{text}```
'''

response = get_completion(user_prompt=prompt, system_role=system_role, model=model)
summarize_message(response)


The assistant role finished 
♢ with "stop" at: 2024-05-28 20:57:46
The model used was: llama-3-sonar-large-32k-online
♢ The message was: 

Analyzing the text, it appears to be a prompt or a question asking for advice on how to make the first $10,000 on Faire, a wholesale marketplace platform. The prompt provides four considerations to guide the response:

1. **Clear Rationale**: The response should clearly explain why using Faire is a viable option for building a business.
2. **Three Detailed Steps**: The response should outline three specific steps to take next, backed by research, to achieve the goal of making the first $10,000 on Faire.
3. **Review Before Responding**: The respondent is advised to take time to review their answer before submitting it, ensuring it meets the requirements.
4. **Time Needed for Review**: The respondent is asked to estimate the extra time needed for this review process.

To provide a comprehensive response, I would need to understand the specifics of Fai

In [18]:
pretty_print_markdown(response)

Analyzing the text, it appears to be a prompt or a question asking for advice on how to make the first $10,000 on Faire, a wholesale marketplace platform. The prompt provides four considerations to guide the response:

1. **Clear Rationale**: The response should clearly explain why using Faire is a viable option for building a business.
2. **Three Detailed Steps**: The response should outline three specific steps to take next, backed by research, to achieve the goal of making the first $10,000 on Faire.
3. **Review Before Responding**: The respondent is advised to take time to review their answer before submitting it, ensuring it meets the requirements.
4. **Time Needed for Review**: The respondent is asked to estimate the extra time needed for this review process.

To provide a comprehensive response, I would need to understand the specifics of Faire's platform, its target market, and the types of products or services being sold. However, based on general knowledge of wholesale marketplaces, here are some potential strategies to make the first $10,000 on Faire:

1. **Niche Product Selection**: Identify a profitable niche with low competition and high demand. Conduct market research to find products that align with Faire's target market and have a clear sales potential.
2. **Optimize Product Listings**: Create high-quality product listings with detailed descriptions, high-quality images, and competitive pricing. Ensure product listings are optimized for search on Faire's platform to increase visibility.
3. **Effective Marketing and Promotion**: Develop a marketing strategy to reach potential customers, such as social media advertising, email marketing, or influencer partnerships. Utilize Faire's promotional tools, such as sales and discounts, to drive sales.

To make the first $10,000 on Faire, it's essential to have a clear understanding of the platform's fees, payment terms, and shipping logistics. Additionally, building relationships with suppliers, managing inventory, and providing excellent customer service are crucial for long-term success.

In terms of the review process, I would estimate an extra 30 minutes to an hour to review and refine the response, ensuring it meets the considerations outlined in the prompt. This time would be spent researching Faire's platform, refining the three detailed steps, and editing the response for clarity and coherence.

### Few shot

In [7]:
# when zero shot prompting produces insufficient response 
system_role = '''you are an experienced entrepreneur and assess knowledge for the 
probability and likelihood of generating revenue.
Your access to intelligence is comprehensive, so you present complex concepts
in college level English with clear structure and in concise, simple terms.
You avoid polemny, especially political or social, especially about
equality, race, religion, creed, genetics, biology, age, and environmental issues.
You never need to apologize and are forthright in presenting the facts
because you really do have full access to the source data available to you.
You review your responses to avoid repetition, and explain logically how they were generated.
You advise as an entrepreneur intent on creating self-sustaining businesses
which support people in finding a place to belong and be in service
'''

prompt = f'''
Your responses follow a consistent format.

[entrepreneur]: Teach me about making money with Faire

[investor]: Find products which are delightful and easy to fulfill;\
1. **Handmade Jewelry from Colombia**\
Colombia is known for its high-quality handmade jewelry, but it is not commonly found on Faire. I would source handmade jewelry from Colombian suppliers on Faire and market them on Instagram as unique, high-quality, and ethically sourced products.\
2. **Natural Skincare Products from Taiwan**\
Taiwan is known for its natural skincare products, but they are not commonly found on Faire. I would source natural skincare products from Taiwanese suppliers on Faire and market them on Instagram as natural, organic, and effective products.\

[entrepreneur]: Teach me about making money with Shopify
'''

response = get_completion(user_prompt=prompt, system_role=system_role, model=model)
summarize_message(response)

### Few shot json prompt

In [8]:
# another few shots; this one interesting for its decent response 

system_role = '''You are an experienced solutions engineer 
providing examples of recent projects using the STAR framework.
You assess your experience  for the 
probability and likelihood of generating revenue.
Your access to intelligence is comprehensive, so you present complex concepts
in college level English with clear structure and in concise, simple terms.
You avoid polemny, especially political or social, especially about
equality, race, religion, creed, genetics, biology, age, and environmental issues.
You never need to apologize and are forthright in presenting the facts
because you really do have full access to the source data available to you.
You review your responses to avoid repetition, and explain logically how they were generated.
You advise as a gifted story-teller building self-sustaining businesses
which support people in finding a place to belong and to be in service
'''

messages = [
    {
        "role": "system",
        "content": system_role
    },
    {
        "role": "user",
        "content": "Topic: Solutions Engineering, Sales"
    },
    {
        "role": "assistant",
        "content": "Here is an example of describing a solutions engineering project in sales using the STAR framework:"
    },
    {
        "role": "user",
        "content": "Topic: Oracle EPM implementations, Infrastructure Manager"
    }
]



    
response = get_completion_json(user_prompt=messages, system_role=system_role, model=model)

In [11]:
summarize_message(response)

The assistant role finished 
♢ with "stop" at: 2024-05-27 20:35:02
The model used was: llama-3-sonar-large-32k-online
♢ The message was: 

As a seasoned solutions engineer, I have had the privilege of working on several Oracle EPM implementations, leveraging my expertise in infrastructure management to ensure seamless integration and optimal performance. Here, I'll share a recent project that exemplifies my skills in this area, using the STAR framework to provide a concise and structured overview.

**S - Situation**
A global manufacturing company, with operations spanning across multiple continents, was struggling to streamline its financial planning and reporting processes. The company's existing EPM system was outdated, leading to inefficiencies, inaccuracies, and a lack of visibility into its financial performance. The management team recognized the need for a modern, cloud-based EPM solution that could integrate with its existing ERP system and provide real-time insights into its f

The assistant role finished 
♢ with "stop" at: 2024-05-27 20:35:02
The model used was: llama-3-sonar-large-32k-online
♢ The message was: 

As a seasoned solutions engineer, I have had the privilege of working on several Oracle EPM implementations, leveraging my expertise in infrastructure management to ensure seamless integration and optimal performance. Here, I'll share a recent project that exemplifies my skills in this area, using the STAR framework to provide a concise and structured overview.

**S - Situation**
A global manufacturing company, with operations spanning across multiple continents, was struggling to streamline its financial planning and reporting processes. The company's existing EPM system was outdated, leading to inefficiencies, inaccuracies, and a lack of visibility into its financial performance. The management team recognized the need for a modern, cloud-based EPM solution that could integrate with its existing ERP system and provide real-time insights into its financial operations.

**T - Task**
My task was to design and implement an Oracle EPM cloud solution that would meet the company's financial planning, reporting, and analytics requirements. This involved assessing the company's current EPM infrastructure, identifying areas for improvement, and developing a customized solution that would integrate with its existing ERP system.

**A - Action**
I worked closely with the company's finance team to understand its business requirements and developed a comprehensive implementation plan. This involved:

1. **Assessing the current EPM infrastructure**: I conducted a thorough analysis of the company's existing EPM system, identifying areas of inefficiency and opportunities for improvement.
2. **Designing a customized EPM solution**: I designed a tailored EPM solution that would meet the company's specific financial planning, reporting, and analytics requirements. This involved configuring the Oracle EPM cloud platform to integrate with the company's existing ERP system.
3. **Implementing the EPM solution**: I led the implementation of the Oracle EPM cloud solution, working closely with the company's IT team to ensure a seamless integration with its existing infrastructure.
4. **Providing training and support**: I provided comprehensive training and support to the company's finance team, ensuring that they were equipped to use the new EPM solution effectively.

**R - Result**
The implementation of the Oracle EPM cloud solution resulted in significant improvements to the company's financial planning and reporting processes. The company was able to:

1. **Streamline financial planning**: The Oracle EPM cloud solution enabled the company to streamline its financial planning processes, reducing the time and effort required to develop and manage budgets and forecasts.
2. **Improve reporting accuracy**: The solution provided real-time insights into the company's financial performance, enabling more accurate and timely reporting.
3. **Enhance visibility**: The company gained greater visibility into its financial operations, enabling more informed decision-making and improved financial management.

Overall, this project demonstrates my ability to design and implement effective EPM solutions that meet the specific needs of my clients. My expertise in infrastructure management and Oracle EPM cloud ensures that my clients can leverage the full potential of their EPM systems, driving business success and growth.

## Specify steps required

In [21]:
# steps, tasks

system_role = '''
You are a Solutions Engineer with expertise in HTML5, Tailwind CSS, Python, and prompt engineeering. 
structure a response using the following steps

Step 1. Text is provided you in triple quotes
Step 2. Translate the text into English, if necessary, then provide a single sentance summary.
Step 3. Extract the main points from the original text, into a Python list.
Step 4. From the original text, output a json object with appropriate keys.

Separate your steps with a right-arrow emoji and a line break
'''

text = '''
et’s be real: the only reason you even know what a ceramic coating is has to do with a social media post by some guy you don’t even really know. It’s flashy, it’s cool, it’s the latest craze in car care that everyone, and no one, is talking about.

If you’re like us, your curiosity was piqued by that mysterious, often pricey fluid that comes in a cute little bottle and is apparently not made of plates (we checked). What ceramic coating actually is is “a long-term nanoscopic exterior automotive paint treatment and protectant that is applied in a liquid form and cures to form a hard layer on top of the paint. Essentially, it's the candy shell that protects the paint's delicious chocolate center.”
Everything You’ll Need To Ceramic Coat a Car
Ceramic coating a car is an intermediate task that should take four hours to apply, but 24 hours to cure. There’s nothing worse than attempting to start a project without everything you need. Make sure you’ve at least got this stuff.

Safety First
Gloves to protect your hands, which will be closest to the product
Long-sleeve shirt and pants. You have skin elsewhere too, cover it as much as possible
A mask or ventilator. You do NOT want to breathe this stuff in
Tool and Parts
Paper towels
3-plus microfiber towels
A wash bucket (or two)
Car soap
Surface prep
Car wash sponge. The mitts are a nice splurge, too
Ceramic coating kit
How to Ceramic Coat a Car, Step by Step
1. Park the Car Somewhere Appropriate
Make sure to work in a covered, well-ventilated, shaded space. Seriously, don’t breathe this stuff in, it’s not air and it smells bad.

2. Wash and Dry the Car
Use the two-bucket method.
3. Prep the Surface
Surface preparation varies depending on how new/well you’ve taken care of your paint. This is arguably the most important step, as a ceramic coating on a poor finish will only seal in the imperfections. You simply are going to get the best results if you start with the best foundation. 

Bare minimum, you’ll want to use a surface preparation solution to remove anything that will get in between the paint and the ceramic coating. A product such as Mother’s CMX Surface Prep is great for removing unwanted surface contaminants. If you wanna go all the way, you’ll want to take a machine polisher to the surface to get it all smooth and glossy.

If the surface of the car is hot, let it cool to between 50 and 70 degrees Fahrenheit so you can apply the coating correctly.

4. Apply the Ceramic Coat
Read the instructions that came with your kit, as they can vary. Here’s a general guide:

Apply the ceramic coating to the provided applicator as directed.
Using the coated applicator, apply the product to the vehicle surface. Work in small areas, and apply in straight lines.
Allow the product to sit as directed, usually 30 to 90 seconds.
5. Wipe Off the Ceramic Coat
With a fresh, clean microfiber towel, wipe off the product.

6. Let It Cure
Once you’ve coated everything you want to coat, be sure to let the product cure for 24 hours. You don’t want anything—even water—to get on the car during this time. Plan ahead!

Ceramic Coating FAQs
Q: Can you mess up ceramic coating?
A: Yes. You won’t mess up the paint, but ceramic coatings are much harder to remove than wax. Take your time with the application process and do it right.

Q: What is ceramic coating made of?
A: Everyone has their proprietary formulation, but the main ingredient is silica dioxide (SiO2).

Q: How long does ceramic coating last?
A: With proper prep, application, and a quality coating formula, a ceramic coating could last 2-5 years.

Q: Can you wax over ceramic coating?
A: Technically yes, but don’t. The short story is that the ceramic coating itself is much better at repelling water and other contaminants than the wax, which will sit on top of the coating. Just keep the car clean and use a ceramic booster spray product every so often.

Q: Does ceramic coating need maintenance?
A:  Everything needs maintenance! Keeping your car washed will help the coating perform at its best. 

Q: How long until you can wash your car after ceramic coating?
A: The general consensus is that you should wait at least seven days before washing your car or driving in the rain, as the coating will still be curing after the initial 24-hour period.
'''

prompt = '''
Describe stages of using ceramic coating on an antique vehicle and identify missing steps
"""{$text}"""
'''

response = get_completion(user_prompt=prompt, system_role=system_role, model=model)


## Display

In [22]:
summarize_message(response)

The assistant role finished 
♢ with "stop" at: 2024-05-28 21:59:09
The model used was: llama-3-sonar-large-32k-online
♢ The message was: 

→ **Text Translation and Summary**
The text is in English, so no translation is needed. The summary is: The text discusses the process of applying ceramic coating to a vehicle, including preparation, polishing, and application steps, as well as considerations for old school single stage lacquer paint and the importance of proper surface preparation.

→ **Main Points**
- Preparation involves cleaning, foaming, rinsing, and drying the vehicle.
- Polishing and paint correction may be necessary before applying ceramic coating.
- Ceramic coating can be applied to old school single stage lacquer paint, but it may require special care.
- Proper surface preparation is crucial for the effectiveness and longevity of the ceramic coating.
- Regular maintenance, including washing and reapplication, is necessary to maintain the coating.

→ **JSON Object**
```json

In [24]:
pretty_print_markdown(response)

→ **Text Translation and Summary**
The text is in English, so no translation is needed. The summary is: The text discusses the process of applying ceramic coating to a vehicle, including preparation, polishing, and application steps, as well as considerations for old school single stage lacquer paint and the importance of proper surface preparation.

→ **Main Points**
- Preparation involves cleaning, foaming, rinsing, and drying the vehicle.
- Polishing and paint correction may be necessary before applying ceramic coating.
- Ceramic coating can be applied to old school single stage lacquer paint, but it may require special care.
- Proper surface preparation is crucial for the effectiveness and longevity of the ceramic coating.
- Regular maintenance, including washing and reapplication, is necessary to maintain the coating.

→ **JSON Object**
```json
{
    "ceramic_coating_process": {
        "preparation": [
            "cleaning",
            "foaming",
            "rinsing",
            "drying"
        ],
        "polishing_and_paint_correction": [
            "necessary_for_old_paint",
            "special_care_for_lacquer_paint"
        ],
        "application": [
            "apply_ceramic_coating",
            "regular_maintenance"
        ],
        "considerations": [
            "proper_surface_preparation",
            "regular_washing_and_reapplication"
        ]
    }
}
```